In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/1.page'):
    shutil.rmtree('./file/1.page')

os.makedirs('./file/1.page')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = input_.loc[a, 'Url']

            dict_param = {}
            list_param = url.split('?')[1].split('&') if '?' in url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]
            
            dict_param.pop('start', '')
            dict_param.pop('num', '')
            
            url = url.split('?')[0] + '?'
            for key, value in dict_param.items():
                url += f'{key}={value}&'

            b = 0
            while True:
                b += 1
                try:
                    resp = requests.get(url[:-1],
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))
            
            # = = = = = = = = = = = = = = =

            count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())

            # = = = = = = = = = = = = = = =

            if count <= 0:
                raise

            # = = = = = = = = = = = = = = =

            output = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                    'Url': url[:-1],
                                    'SKU Count': count,
                                    'Page': math.ceil(count / 100)}])
            
            output = output.sort_values(by=['No'],
                                        ascending=[True],
                                        ignore_index=True)
            
            output.to_excel(f'''./file/1.page/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/crawler_{input_.loc[a, 'No']}-part_number_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.page'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/1.page/{file}',
              f'''./file/1.page/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[状态：ok，尝试次数：1] - 2.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=Engine%2520Oil%2520Pan&start=0&num=100
[剩余数量：0] - [当前时间：16:49:09]

[状态：ok，尝试次数：1] - 1.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=wheel%2520cylinder&start=0&num=100
[剩余数量：0] - [当前时间：16:49:14]

输出ing...



Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4007.94it/s]

Done ~
